##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Training 和 Serving（使用 TFX 和 Vertex Pipelines）


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 Tensorflow.org 上查看</a> </td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a> </td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">在 Google Cloud Vertex AI Workbench 中运行</a></td>
</table></div>


这个基于笔记本的教程将创建并运行 TFX 流水线，该流水线使用 Vertex AI Training 服务训练 ML 模型并将其发布到 Vertex AI 以便应用。

此笔记本基于我们在[适用于 Vertex Pipelines 的简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)中构建的 TFX 流水线。如果您还未阅读该教程，应在继续使用此笔记本之前阅读它。

您可以使用 AutoML 在 Vertex AI 上训练模型，也可以使用自定义训练。在自定义训练中，您可以选择许多不同的机器类型来支持您的训练作业、启用分布式训练、使用超参数调整以及使用 GPU 进行加速。

您还可以将经过训练的模型部署到 Vertex AI 模型并创建端点，从而应用预测请求。

在本教程中，我们将配合使用 Vertex AI Training 和自定义作业来训练 TFX 流水线中的模型。我们还将部署模型以使用 Vertex AI 应用预测请求。


此笔记本旨在于 [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) 或 [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks) 上运行。如果您不使用其中之一，只需单击上面的“在 Google Colab 中运行”按钮即可。

## 设置

如果您已经完成[适用于 Vertex Pipelines 的简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)，您将拥有一个 GCP 工作项目和一个 GCS 存储桶，这就是本教程所需要的全部内容。如果您错过了初步教程，请先阅读它。

### 安装 python 软件包

我们将安装所需的 Python 软件包（包括 TFX 和 KFP），以编写 ML 流水线并将作业提交到 Vertex Pipelines。

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

### 卸载 Shapely

TODO(b/263441833) 这是避免 ImportError 的临时解决方案。最终，应该通过支持最新版本的 Bigquery 来处理，而不是卸载其他额外的依赖项。

In [ ]:
!pip uninstall shapely -y

#### 是否已重新启动运行时？

如果您使用的是 Google Colab，则在首次运行上面的代码单元时必须重新启动运行时，方法是单击上面的“重新启动运行时”按钮或使用“运行时 &gt; 重新启动运行时...”菜单。这样做的原因是 Colab 加载软件包的方式。

如果您不在 Colab 上，可以使用以下代码单元重新启动运行时。

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### 登录 Google 获取此笔记本

如果您在 Colab 上运行此笔记本，请使用您的用户帐户进行身份验证：

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**如果您在 AI Platform Notebooks 上**，请在运行下一部分之前通过 Google Cloud 进行身份验证，方法是运行

```sh
gcloud auth login
```

运行位置为**终端窗口** （可通过菜单中的**文件** &gt; **新建**打开）。您只需对每个笔记本实例执行一次此操作。

检查软件包版本。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### 设置变量

我们将在下面设置一些用于自定义流水线的变量。所需信息如下：

- GCP 项目 ID。请参阅[找出项目 ID](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects)。
- 用于运行流水线的 GCP 区域。有关 Vertex Pipelines 在其中可用的区域的更多信息，请参阅 [Vertex AI 位置指南](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability)。
- 用于存储流水线输出的 Google Cloud Storage 存储桶。

**在下面的代码单元中输入所需值，然后再运行它**。


In [ ]:
GOOGLE_CLOUD_PROJECT = ''     # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''      # <--- ENTER THIS
GCS_BUCKET_NAME = ''          # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

设置 `gcloud` 以使用您的项目。

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-vertex-training'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Name of Vertex AI Endpoint.
ENDPOINT_NAME = 'prediction-' + PIPELINE_NAME

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

### 准备示例数据

我们将使用与[简单 TFX 流水线教程](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)相同的 [Palmer Penguins 数据集](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)。

此数据集中有四个数字特征，这些特征已标准化为具有范围 [0,1]。我们将建立一个预测企鹅 `species` 的分类模型。

我们需要创建我们自己的数据集副本。因为 TFX ExampleGen 从目录中读取输入，所以我们需要在 GCS 上创建一个目录并将数据集复制到其中。

In [ ]:
!gsutil cp gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv {DATA_ROOT}/

快速浏览一下 CSV 文件。

In [ ]:
!gsutil cat {DATA_ROOT}/penguins_processed.csv | head

## 创建流水线

我们的流水线将与我们在[适用于 Vertex Pipelines 的简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)中创建的流水线非常相似。该流水线将包含三个组件：CsvExampleGen、Trainer 和 Pusher。但是，我们将使用特殊的 Trainer 和 Pusher 组件。Trainer 组件会将训练工作负载移动到 Vertex AI，Pusher 组件则会将经过训练的 ML 模型发布到 Vertex AI 而不是文件系统。

TFX 提供一个特殊的 `Trainer`，用于将训练作业提交给 Vertex AI Training 服务。我们所要做的就是在扩展模块中使用 `Trainer`（而非标准 `Trainer` 组件）以及一些必需的 GCP 参数。

在本教程中，我们将首先使用多个 CPU 运行 Vertex AI Training 作业，然后再使用一个 GPU 来运行。

TFX 还提供一个特殊的 `Pusher`，用于将模型上传到 *Vertex AI 模型*。`Pusher` 还将创建 *Vertex AI 端点*资源以应用在线预测。要了解有关 Vertex AI 提供的在线预测的更多信息，请参阅 [Vertex AI 文档](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)。

### 编写模型代码。

该模型本身与[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中的模型几乎相似。

我们将添加可创建 [TensorFlow 分布策略](https://tensorflow.google.cn/guide/distributed_training)的 `_get_distribution_strategy()` 函数，其用于 `run_fn`，以便在 GPU 可用时使用 MirroredStrategy。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple and
# slightly modified run_fn() to add distribution_strategy.

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
        for feature in _FEATURE_KEYS
    }, _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://tensorflow.google.cn/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# NEW: Read `use_gpu` from the custom_config of the Trainer.
#      if it uses GPU, enable MirroredStrategy.
def _get_distribution_strategy(fn_args: tfx.components.FnArgs):
  if fn_args.custom_config.get('use_gpu', False):
    logging.info('Using MirroredStrategy with one GPU.')
    return tf.distribute.MirroredStrategy(devices=['device:GPU:0'])
  return None


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  # NEW: If we have a distribution strategy, build a model in a strategy scope.
  strategy = _get_distribution_strategy(fn_args)
  if strategy is None:
    model = _make_keras_model()
  else:
    with strategy.scope():
      model = _make_keras_model()

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

将模块文件复制到可从流水线组件进行访问的 GCS。

否则，您可能需要构建包含模块文件的容器映像并使用该映像来运行流水线和 AI Platform Training 作业。

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### 编写流水线定义

我们将定义一个函数来创建 TFX 流水线。它具有与[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中相同的三个组件，但我们将在 GCP 扩展模块中使用 `Trainer` 和 `Pusher` 组件。

`tfx.extensions.google_cloud_ai_platform.Trainer` 的行为类似于普通 `Trainer`，只是将模型训练的计算移动到云。它在 Vertex AI Training 服务中启动自定义作业，而且编排系统中的 Trainer 组件会一直等待，直到 Vertex AI Training 作业完成。

`tfx.extensions.google_cloud_ai_platform.Pusher` 使用经过训练的模型创建 Vertex AI 模型和 Vertex AI 端点。


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, endpoint_name: str, project_id: str,
                     region: str, use_gpu: bool) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Configuration for Vertex AI Training.
  # This dictionary will be passed as `CustomJobSpec`.
  vertex_job_spec = {
      'project': project_id,
      'worker_pool_specs': [{
          'machine_spec': {
              'machine_type': 'n1-standard-4',
          },
          'replica_count': 1,
          'container_spec': {
              'image_uri': 'gcr.io/tfx-oss-public/tfx:{}'.format(tfx.__version__),
          },
      }],
  }
  if use_gpu:
    # See https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype
    # for available machine types.
    vertex_job_spec['worker_pool_specs'][0]['machine_spec'].update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })

  # Trains a model using Vertex AI Training.
  # NEW: We need to specify a Trainer for GCP with related configs.
  trainer = tfx.extensions.google_cloud_ai_platform.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5),
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.TRAINING_ARGS_KEY:
              vertex_job_spec,
          'use_gpu':
              use_gpu,
      })

  # NEW: Configuration for pusher.
  vertex_serving_spec = {
      'project_id': project_id,
      'endpoint_name': endpoint_name,
      # Remaining argument is passed to aiplatform.Model.deploy()
      # See https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#deploy_the_model
      # for the detail.
      #
      # Machine type is the compute resource to serve prediction requests.
      # See https://cloud.google.com/vertex-ai/docs/predictions/configure-compute#machine-types
      # for available machine types and acccerators.
      'machine_type': 'n1-standard-4',
  }

  # Vertex AI provides pre-built containers with various configurations for
  # serving.
  # See https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
  # for available container images.
  serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest'
  if use_gpu:
    vertex_serving_spec.update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })
    serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-6:latest'

  # NEW: Pushes the model to Vertex AI.
  pusher = tfx.extensions.google_cloud_ai_platform.Pusher(
      model=trainer.outputs['model'],
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
              serving_image,
          tfx.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
            vertex_serving_spec,
      })

  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

## 在 Vertex Pipelines 上运行流水线。

我们将使用 Vertex Pipelines 来运行流水线，就像我们在[适用于 Vertex Pipelines 的简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)中所做的那样。

In [ ]:
# docs_infra: no_execute
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # We will use CPUs only for now.
        use_gpu=False))

生成的定义文件可以使用 `google-cloud-aiplatform` 软件包中的 Google Cloud aiplatform 客户端提交。

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

现在，您可以访问上面输出中的链接或访问 [Google Cloud Console](https://console.cloud.google.com/) 中的“Vertex AI &gt; 流水线”来查看进度。

## 使用预测请求进行测试

流水线完成后，您将在“Vertex AI &gt; 端点”中的端点之一处找到*已部署*的模型。我们需要知道端点的 ID 才能向新端点发送预测请求。这与我们上面输入的*端点名称*不同。您可以在 <code>Google Cloud Console</code> 的<a>“端点”页面</a>找到 ID，它看起来是一个很长的编号。

**设置下面的 ENDPOINT_ID，然后再运行它。**


In [ ]:
ENDPOINT_ID=''     # <--- ENTER THIS
if not ENDPOINT_ID:
    from absl import logging
    logging.error('Please set the endpoint id.')

我们使用相同的 aiplatform 客户端向端点发送请求。我们将发送企鹅物种分类的预测请求。输入是我们使用的四个特征，模型将返回三个值，因为我们的模型为每个物种输出一个值。

例如，以下具体示例在索引“2”处具有最大值，并将打印“2”。


In [ ]:
# docs_infra: no_execute
import numpy as np

# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
# Our model expects 4 feature inputs and produces 3 output values for each
# species. Note that the output is logit value rather than probabilities.
# See the model code to understand input / output structure.
instances = [{
    'culmen_length_mm':[0.71],
    'culmen_depth_mm':[0.38],
    'flipper_length_mm':[0.98],
    'body_mass_g': [0.78],
}]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances)

# Uses argmax to find the index of the maximum value.
print('species:', np.argmax(response.predictions[0]))

有关在线预测的详细信息，请访问 <code>Google Cloud Console</code> 中的<a>“端点”页面</a>。您可以找到有关发送示例请求和更多资源链接的指南。

## 使用 GPU 运行流水线


Vertex AI 支持使用各种机器类型进行训练，包括对 GPU 的支持。有关可用选项，请参阅[机器规格参考](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype)。

我们已经定义了流水线来支持 GPU 训练。我们需要做的就是将 `use_gpu` 标志设置为 True。之后将会使用包含一个 NVIDIA_TESLA_K80 的机器规格创建流水线，我们的模型训练代码将使用 `tf.distribute.MirroredStrategy`。

请注意，`use_gpu` 标志不是 Vertex 或 TFX API 的一部分。它只用于控制本教程中的训练代码。

In [ ]:
# docs_infra: no_execute
runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # Updated: Use GPUs. We will use a NVIDIA_TESLA_K80 and 
        # the model code will use tf.distribute.MirroredStrategy.
        use_gpu=True))

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

现在，您可以访问上面输出中的链接或访问 [Google Cloud Console](https://console.cloud.google.com/) 中的“Vertex AI &gt; 流水线”来查看进度。

## 清理

您已在本教程中创建了 Vertex AI 模型和端点。请首先转到[端点](https://console.cloud.google.com/vertex-ai/endpoints)并从端点*取消部署*模型，以此方式删除这些资源以避免任何不必要的费用。然后，您可以单独删除端点和模型。